In [68]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set up the directories
current_dir = os.getcwd()
code_dir = os.path.dirname(current_dir)
project_dir = os.path.dirname(code_dir)
data_dir = os.path.join(project_dir, 'data')
raw_data_dir = os.path.join(data_dir, 'raw_data')
ad_hoc_data_dir = os.path.join(data_dir, 'ad_hoc_data')
processed_data_dir = os.path.join(data_dir, 'processed_data')

# LOAD THE DATA
# Index are ENHANCE_ID
# Food items match
file_path = os.path.join(ad_hoc_data_dir, 'food_items_match.xlsx')
food_items_match = pd.read_excel(file_path, index_col=4)

# Food items prices
file_path = os.path.join(raw_data_dir, '5.Cambodia_FNG_price_data.xlsx')
cambodia_fng_prices = pd.read_excel(file_path, sheet_name='price_data', index_col=0)

# Edible factor
file_path = os.path.join(raw_data_dir, '4.CSES_Cambodia_foodlist_matching_v2.xlsx')
edible_factors = pd.read_excel(file_path, sheet_name='FCT_matches', index_col=0)

# DATA PROCESSING
# New column 'Cost (KHR/g)' is 'Price_100g' divided by 100
cambodia_fng_prices['Cost (KHR/g)'] = cambodia_fng_prices['Price_100g'] / 100
# Add new column 'EDIBLE' to cambodia_fng_prices from edible_factors (merge)
cambodia_fng_prices = pd.merge(cambodia_fng_prices, edible_factors[['EDIBLE']], left_index=True, right_index=True, how='left')
# New column 'Cost wg (KHR/g)' is 'Cost (KHR/g)' divided by 'EDIBLE'
cambodia_fng_prices['Cost wg (KHR/g)'] = cambodia_fng_prices['Cost (KHR/g)'] / cambodia_fng_prices['EDIBLE']
# Change 'EDIBLE' to 'Edible factor'
cambodia_fng_prices.rename(columns={'EDIBLE': 'Edible factor'}, inplace=True)
# Drop columns 'Price_100g', 'Extraction rate', 'ENHANCE_ID'
cambodia_fng_prices.drop(columns=['Price_100g', 'Price_date'], inplace=True)
# Merge to build the final whole dataframe 
food_prices_df = pd.merge(food_items_match, cambodia_fng_prices, left_index=True, right_index=True, how='outer')
food_prices_df.reset_index(inplace=True)
# Remove duplicated rows
food_prices_df.drop_duplicates(inplace=True)
# Drop columns 'food_name_english', 'food_id', 'group_id', 'Food group', 'fbs_item', 'Extraction rate'
food_prices_df.drop(columns=['food_name_english', 'food_id', 'group_id', 'Food group', 'Extraction rate', 'fbs_item'], inplace=True)
# One dataframe for each region, stored in a dictionary
region_names = food_prices_df['Region_Name'].unique()

food_prices_by_region = {}

for region_name in region_names:
    df = food_prices_df[food_prices_df['Region_Name'] == region_name]
    # Delete index by considering it as a column
    #df.reset_index(inplace=True)
    # Rename region_name to keep only number
    region_name = region_name.split(' ')[0:2]
    region_name = ' '.join(region_name)
    # Delete rows with NaN values for 'Food names'
    df = df.dropna(subset=['Food names'], inplace=False)
    # Merge food_items_match and df conserving all 'Food names' from food_items_match ('Food names' is not the index)
    df = pd.merge(food_items_match, df, on='Food names', how='left')
    # Drop columns 'Food group', 'Extraction rate', 'fbs_item'
    df.drop(columns=['Food group', 'Extraction rate', 'fbs_item'], inplace=True)
    # Resort the first three columns to have 'ENHANCE_ID', 'food_id', 'group_id', 'Food names' at the beginning in this order
    df = df[['ENHANCE_ID', 'food_id', 'group_id', 'Food names'] + [col for col in df.columns if col not in ['ENHANCE_ID', 'food_id', 'group_id', 'Food names']]]
    
    food_prices_by_region[region_name] = df
    
region_numbers = list(food_prices_by_region.keys())

# Sort by number after 'Region ' (e.g. 'Region 1', 'Region 2', ...)
region_numbers.sort(key=lambda x: int(x.split(' ')[1]))

# Save the data in a single Excel file with one sheet per region adjusting column width to fit the longest string in each column
file_path = os.path.join(processed_data_dir, 'food_prices.xlsx')

with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    for region_number in region_numbers:
        df = food_prices_by_region[region_number]
        df.to_excel(writer, sheet_name=region_number, index=False)
        worksheet = writer.sheets[region_number]
        for i, col in enumerate(df.columns):
            # find length of column i
            column_len = df[col].astype(str).str.len().max()
            # Setting the length if the column header is larger
            column_len = max(column_len, len(col))
            # set the column length
            worksheet.set_column(i, i, column_len)